# Agentic RAG Playground by LlamaIndex

In [1]:
import datasets
from llama_index.core.schema import Document

from src.tools import GuestInfoRetrieverTool
from src.retriever import query_guest_agent

In [2]:
# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        text="\n".join([
            f"Name: {guest_dataset['name'][i]}",
            f"Relation: {guest_dataset['relation'][i]}",
            f"Description: {guest_dataset['description'][i]}",
            f"Email: {guest_dataset['email'][i]}"
        ]),
        metadata={"name": guest_dataset['name'][i]}
    )
    for i in range(len(guest_dataset))
]

In [3]:
# Initialize the tool
guest_info_tool = GuestInfoRetrieverTool(docs)

In [ ]:
# Example query
question = "Tell me about our guest named 'Lady Ada Lovelace'."

# Call the function using default parameters
response = await query_guest_agent(docs, question)

# Generalize the response handling (same as before)
if response and response.response:
    print("🎩 Alfred's Response:")
    print(response.response)
else:
    print("🎩 Alfred did not provide a response.")

In [ ]:
# Another example query
question = "Who is Charles Babbage?"

# Call the function with custom parameters
response = await query_guest_agent(
    docs,
    question,
    llm_model="phi3:mini", # Use a different LLM model
    system_prompt="You are a helpful AI assistant that provides information about people." # Use a different system prompt
)

# Generalize the response handling (same as before)
if response and response.response:
    print("🎩 Alfred's Response:")
    print(response.response)
else:
    print("🎩 Alfred did not provide a response.")

## Give Your Agent Access to the Web

In [4]:
from src.tools import DDGSearchTool

# Create an instance of the DDGSearchTool class
ddst = DDGSearchTool()

In [ ]:
# Use the search_tool method of the instance
response = ddst.search_tool("Who's the founder of Turkish Republic?")
print(response)

In [5]:
from src.tools import WeatherInfoTool

# Initialize the tool
# The DuckDuckGoSearchTool class already initializes the FunctionTool internally.
# We just need to create an instance of the class.
wit = WeatherInfoTool()

In [6]:
from src.tools import HubStatsTool

hst = HubStatsTool()

In [ ]:
# Example usage
response = hst.get_hub_stats("facebook")
print(response)

## Integrate weather_info_tool and hub_stats_tool to Alfred

In [32]:
async def generalized_multi_step_agent(query, llm, tools):
    # Step 1: Plan the execution
    planning_prompt = f"""
    Analyze the following user query and create a plan to answer it using the available tools.
    Available tools:
    - dd_search_tool: Use to search the internet for general information.
    - hub_stats_tool: Use to find the most downloaded model for a Hugging Face author.
    - weather_info_tool: Use to get weather information for a location.

    Output the plan as a JSON array of steps. Each step should have:
    - "task": A description of the task for this step.
    - "tool": The name of the tool to use (from the available tools).
    - "tool_input": The input for the tool (e.g., a search query, an author name, a location).

    Example output format:
    ```json
    [
      {{
        "task": "Example task",
        "tool": "example_tool",
        "tool_input": {{"key": "value"}}
      }}
    ]
    ```

    User query: {query}
    """
    plan_response = await llm.acomplete(planning_prompt)

    # Extract the JSON plan from the LLM's output
    plan_text = plan_response.text.strip()
    json_start = plan_text.find("```json")
    json_end = plan_text.find("```", json_start + 7) # Find the closing ``` after the opening one

    if json_start != -1 and json_end != -1:
        # Extract the text between the triple backticks
        json_string = plan_text[json_start + 7:json_end].strip()
        try:
            plan = json.loads(json_string)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON plan: {e}")
            print(f"Extracted JSON string: {json_string}")
            return "Error: Could not parse the planning response from the LLM."
    else:
        print("Error: Could not find JSON block in LLM output.")
        print(f"LLM output: {plan_text}")
        return "Error: Could not find the planning response in the expected format."

    tool_results = {}
    
    # Step 2: Execute the plan
    for step in plan:
        task = step["task"]
        tool_name = step["tool"]
        tool_input = step["tool_input"]

        if tool_name in tools:
            tool_instance = tools[tool_name]
            try:
                result = await tool_instance.acall(**tool_input) # Use **tool_input to unpack dict
                # Extract the text content from the ToolOutput object
                tool_results[task] = result.content
                # print(f"Executed '{task}': {result.content}") # Comment out or remove this line
            except Exception as e:
                tool_results[task] = f"Error executing tool {tool_name}: {str(e)}\n" # Added newline for better formatting
                # print(f"Error executing '{task}': {e}") # Comment out or remove this linets[task] = result.content
                print(f"Executed '{task}': {result.content}") # This line prints the execution details
            except Exception as e:
                tool_results[task] = f"Error executing tool {tool_name}: {str(e)}\n" # Added newline for better formatting
                print(f"Error executing '{task}': {e}") 
        else:
            tool_results[task] = f"Error: Tool '{tool_name}' not found."
            print(f"Error: Tool '{tool_name}' not found.")

    # Step 3: Synthesize the information
    synthesis_prompt = f"""
    The user asked: '{query}'.
    You gathered the following information:
    {json.dumps(tool_results, indent=2)}

    Please synthesize this information into a comprehensive answer that addresses all parts of the original question.
    """
    synthesized_answer = await llm.acomplete(synthesis_prompt)

    # Return both the tool results and the synthesized answer
    return {
        "tool_results": tool_results,
        "final_answer": synthesized_answer.text
    }

In [33]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
import traceback
from llama_index.core.tools import FunctionTool
import asyncio

# Initialize the Hugging Face model
llm = Ollama(model="llama3:latest", request_timeout=1200)

search_tool = FunctionTool.from_defaults(
        fn=ddst.search_tool,
        name="dd_search_tool",
        description=(
            "Use this tool to search the internet for general information about a topic, "
            "definitions, explanations, or factual details that are likely to be found on websites. "
            "This is a general-purpose search tool."
        )
    )

weather_info_tool = FunctionTool.from_defaults(
        fn=wit.get_weather_info,
        name="weather_info_tool",
        description="Use this tool ONLY when the user is asking for weather information for a specific location."
    )

hub_stats_tool = FunctionTool.from_defaults(
        fn=hst.get_hub_stats,
        name="hub_stats_tool",
        description=(
            "Use this tool to find statistics about models on the Hugging Face Hub. "
            "Specifically, use this tool to find the most downloaded model for a given author or organization on the Hugging Face Hub."
        )
    )


# Create Alfred with all the tools
alfred = ReActAgent.from_tools(
        [search_tool, weather_info_tool, hub_stats_tool],
        llm=llm,
        verbose=True,
        system_prompt=(
                "You are a helpful assistant that can answer questions by using the provided tools. "
                "When asked a question, first create a plan to answer the question by breaking it down into smaller steps and identifying which tools are needed for each step. "
                "After creating the plan, execute the steps using the tools. "
                "Once you have gathered all the necessary information from the tools, your next thought should be to synthesize the information. "
                "Then, synthesize all the information from the tool observations into a comprehensive and coherent answer that addresses all parts of the user's original question. "
                "Always follow the thought-action-input format for tool use."
            )
    )

In [35]:
# In your main execution:
question = "What is Facebook and what's their most popular model?"

tools_dict = {
     "dd_search_tool": search_tool,
     "hub_stats_tool": hub_stats_tool,
     "weather_info_tool": weather_info_tool
 }

response_data = await generalized_multi_step_agent(question, llm, tools_dict)

print_details = False # Set to False to print only the final answer

if print_details:
    print("--- Execution Details ---")
    for task, result in response_data["tool_results"].items():
        print(f"Executed '{task}': {result}")
    print("-----------------------")

print("🎩 Alfred's Response:")
print(response_data["final_answer"])

🎩 Alfred's Response:
Here's a comprehensive answer to the user's question:

Facebook is a social media platform that allows users to connect with friends, family, and communities of people who share similar interests. The Facebook app features various tools that make it easy to discover new connections, such as Groups, Watch, and Marketplace.

As for their most popular model, according to Hugging Face, the leading author associated with Facebook models, the most downloaded model is indeed "facebook/esmfold_v1" - a remarkable 20,947,347 downloads! This suggests that this particular model has gained significant traction among users.
